## Visión por Computadora - Trabajo Práctico 4

- Diego Braga

### Enunciado

Objetivo:

1. Implementar el detector de fondo naive usando la mediana como estimador. El algoritmo debe recibir el parámetro N (cantidad de frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo.
2. Se deben generar las mascaras de foreground y aplicarlas a los frames para segmentar los objetos en movimiento.
3. Comparar con alguno de los métodos vistos en la practica basados en mezcla de gaussianas.

In [23]:
import cv2
import numpy as np
import random
import psutil
import tracemalloc

Para resolver este ejercicio se definieron dos funciones. Por un lado se creó *background_subtraction_naive* que utiliza la mediana como estimador y por otro *background_subtraction_mog2* que simplemente utiliza la implementación de OpenCV de MOG2. La función *background_subtraction_naive* recibe los parámetros *N* y *recalc_interval*, siendo el primero la cantidad de frames utilizados para calcular la estimación y *recalc_interval* el intervalo de recálculo (medido en número de frames transcurridos). La primera estimación se realiza a los *N* frames y luego cada *recalc_interval* frames. La idea es que cada *recalc_interval* frames se estarán tomando de forma aleatoria *N* frames desde la última actualización para realizar el cálculo de la mediana.

Se utilizan algunas operaciones morfológicas para mejorar el resultado binarizado de la diferencia del frame con la mediana del fondo. En particular:

- MORPH_OPEN: Aplica una erosión seguida de una dilatación. Es útil para reducir ruido.
- MORPH_CLOSE: Aplica una dilatación y luego una erosión. En este caso se utiliza para rellenar pequeños agujeros que puedan quedar en la máscara.

El tamaño de los kernels se obtuvo mediante la experimentación. También se realizaron pruebas haciendo las erosiones y dilataciones por separado, pero no hubo mejoras significativas.

Ambas funciones implementadas devuelven métricas de consumo de CPU y memoria.

In [24]:
def background_subtraction_naive(video_path, N, recalc_interval):
    """
    Función que implementa el algoritmo naive utilizando la mediana como estimador para sustracción del fondo en video.
    
    Args:
    video_path (str): Ruta al video a analizar.
    N (int): Cantidad de frames utilizados para calcular la estimación.
    recalc_interval (int): Intervalo de recálculo (medido en número de frames transcurridos).

    Returns:
    avg_cpu (float): Promedio de uso de CPU.
    memory_peak (float): Uso de memoria (peak).
    """

   # Se lee el video recibido por parámetro
    capture = cv2.VideoCapture(video_path)
    fps = int(capture.get(cv2.CAP_PROP_FPS))
    delay = int(1000 / fps)

    if not capture.isOpened():
        print('Error al abrir video')
        return

    # Inicialización de variables
    frames = []
    interval_counter = 0
    cpu_usage = []
    median_background = None
    kernel_morph_open = np.ones((3, 3), np.uint8)
    kernel_morph_close = np.ones((2, 2), np.uint8)

    # Se define el tamaño de la ventana desde donde se seleccionan los posibles N frames aleatoriamente
    frame_window_size = max(N, recalc_interval)

    # Se activa el rastreo de memoria
    tracemalloc.start()

    while True:
        ret, frame = capture.read()
        if not ret:
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Se almacenan frames para detección del fondo
        frames.append(gray_frame)
        if len(frames) > frame_window_size:
            frames.pop(0)

        # Se recalcula el fondo cada recalc_interval frames o si pasaron los primeros N frames
        if (interval_counter % recalc_interval == 0) or (median_background is None and len(frames) >= N):
            # Se seleccionan N frames aleatoriamente desde el último cálculo
            random_frames = random.sample(frames, min(N, len(frames)))
            # Se calcula la mediana de los frames
            median_background = np.median(random_frames, axis=0).astype(np.uint8)

        # Se comienza a mostrar el video cuando se tienen los frames suficientes para calcular el primer fondo
        if len(frames) >= N:

            # Se resta la mediana al frame
            fg_mask = cv2.absdiff(gray_frame, median_background)

            # Se binariza la resta
            _, fg_mask = cv2.threshold(fg_mask, 35, 255, cv2.THRESH_BINARY)

            # Se aplica apertura y cierre (operaciones morfológicas) para mejorar la máscara (reducción de ruido y relleno de agujeros)
            fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel_morph_open)
            fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel_morph_close)

            # Monitoreo de recursos
            cpu_usage.append(psutil.cpu_percent())

            interval_counter += 1

            cv2.imshow('Original', frame)
            cv2.imshow('Naive (Mediana)', fg_mask)

        # Salir con la tecla 'q'
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break

    # Se obtiene pico de consumo de memoria
    _, memory_peak = tracemalloc.get_traced_memory()
    memory_peak /= 1024 * 1024

    capture.release()
    cv2.destroyAllWindows()
    tracemalloc.stop()

    # Se calcula media de consumo de CPU
    avg_cpu = np.mean(cpu_usage)

    return avg_cpu, memory_peak


In [25]:
def background_subtraction_mog2(video_path):
    """
    Función que utiliza MOG2 de OpenCV para sustracción del fondo en video.
    
    Args:
    video_path (str): Ruta al video a analizar.
    N (int): Cantidad de frames utilizados para calcular la estimación.
    recalc_interval (int): Intervalo de recálculo (medido en número de frames transcurridos).

    Returns:
    avg_cpu (float): Promedio de uso de CPU.
    memory_peak (float): Uso de memoria (peak).
    """

    capture = cv2.VideoCapture(video_path)
    mog2 = cv2.createBackgroundSubtractorMOG2()
    fps = int(capture.get(cv2.CAP_PROP_FPS))
    delay = int(1000 / fps)

    if not capture.isOpened():
        print('Falla al abrir el archivo')
        return

    cpu_usage = []
    tracemalloc.start()

    while True:
        ret, frame = capture.read()
        if not ret:
            break

        # Se aplica MOG2 (OpenCV)
        fg_mask_mog2 = mog2.apply(frame)

        # Se acumulan datos de monitoreo de CPU
        cpu_usage.append(psutil.cpu_percent())

        cv2.imshow('Original', frame)
        cv2.imshow('MOG2', fg_mask_mog2)

        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break

    # Se obtiene pico de consumo de memoria
    _, memory_peak = tracemalloc.get_traced_memory()
    memory_peak /= 1024 * 1024

    capture.release()
    cv2.destroyAllWindows()

    tracemalloc.stop()

    # Se calcula la media de consumo de CPU
    avg_cpu = np.mean(cpu_usage)

    return avg_cpu, memory_peak

In [26]:
# Ruta al video de ejemplo
video_path = 'vtest.avi'

In [27]:
# Se ejecuta el algoritmo naive
naive_avg_cpu, naive_memory_peak = background_subtraction_naive(video_path, N=30, recalc_interval=60)

In [28]:
# Se ejecuta el algoritmo MOG2
mog2_avg_cpu, mog2_memory_peak = background_subtraction_mog2(video_path)

A simple vista se puede observar que el modelo naive, utilizando procesamiento morfológico, presenta menor ruido que MOG2 sin ningún procesamiento. Hay algunas situaciones, como por ejemplo cuando algunas personas quedan paradas mucho tiempo en el mismo lugar, que pueden hacer que sean interpretadas como parte del fondo en algunos intervalos. Esto claramente se puede ajustar aumentando el valor de *N* y a su vez reduciendo el valor de *recalc_interval*. De todas formas seguramente queden artefactos debido a que hay mucha circulación de personas todo el tiempo.

Este video tiene la particularidad de que el fondo es bastante estático, lo cual hace que el modelo *naive* tenga un buen resultado. Sin embargo, MOG2 es más tolerante a dichos cambios y tiene la capacidad de aprender de forma más fluida frame a frame.

De las ejecuciones anteriores también se obtuvieron datos de usos máximo de memoria y promedios de utilización de CPU, cuyos resultados fueron los siguientes:

In [29]:
print(f'Memoria utilizada (peak) por algoritmo naive: { naive_memory_peak } MB')
print(f'Promedio de uso de CPU por algoritmo naive: { naive_avg_cpu } %')

Memoria utilizada (peak) por algoritmo naive: 56.267741203308105 MB
Promedio de uso de CPU por algoritmo naive: 14.181592689295037 %


In [30]:
print(f'Memoria utilizada (peak) por algoritmo MOG2: { mog2_memory_peak } MB')
print(f'Promedio de uso de CPU por algoritmo MOG2: { mog2_avg_cpu } %')

Memoria utilizada (peak) por algoritmo MOG2: 2.9782047271728516 MB
Promedio de uso de CPU por algoritmo MOG2: 18.117861635220123 %


Comparando los dos algoritmos en cuanto a recursos utilizados, se puede notar que la versión *naive* consume más memoria que MOG2. Esto seguramente se deba a que su implementación requiere tener una ventana de frames en memoria para poder calcular su media cuando corresponda. Por otro lado MOG2 no tiene este requisito, si no que realiza los cálculos frame a frame utilizando los datos del histograma.

En cuanto al uso de CPU se puede notar un mayor consumo promedio por parte de MOG2 ya que debe realizar cálculos más complejos frame a frame. De todas formas, es conveniente analizar en qué líneas del código se producen estos picos.

Para esto se utilizó el módulo *line_profiler* que permite obtener tiempos de ejecución a nivel de líneas de código.

In [31]:
# Se carga el módulo
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [32]:
%lprun -f background_subtraction_naive background_subtraction_naive(video_path, N=30, recalc_interval=60)

Timer unit: 1e-07 s

Total time: 89.0021 s
File: C:\Users\Usuario\AppData\Local\Temp\ipykernel_3852\3580087281.py
Function: background_subtraction_naive at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def background_subtraction_naive(video_path, N, recalc_interval):
     2                                               """
     3                                               Función que implementa el algoritmo naive utilizando la mediana como estimador para sustracción del fondo en video.
     4                                               
     5                                               Args:
     6                                               video_path (str): Ruta al video a analizar.
     7                                               N (int): Cantidad de frames utilizados para calcular la estimación.
     8                                               recalc_interval (int): Intervalo de recálculo (m

De este profiling se puede observar que hay varias líneas que son las que consumen más tiempo. Algunas son comunes a ambos algoritmos, como por ejemplo la lectura del video, la conversión a escala de grises y, obviamente, el wait para respetar el frame rate. Pero hay una en particular que se destaca y es la siguiente:

*median_background = np.median(random_frames, axis=0).astype(np.uint8)*

Esta línea es la que calcula la mediana de los *N* backgrounds seleccionados y es clave en el algoritmo. Incluso en las pruebas realizadas localmente se pudo observar en tiempo de ejecución que en el momento del cálculo hay un pequeño lag en el video. Esto podría significar que no sea adecuada su utilización en una aplicación de tiempo real, a menos que se tengan los recursos computacionales adecuados.

In [33]:
%lprun -f background_subtraction_mog2 background_subtraction_mog2(video_path)

Timer unit: 1e-07 s

Total time: 96.4043 s
File: C:\Users\Usuario\AppData\Local\Temp\ipykernel_3852\3278358850.py
Function: background_subtraction_mog2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def background_subtraction_mog2(video_path):
     2                                               """
     3                                               Función que utiliza MOG2 de OpenCV para sustracción del fondo en video.
     4                                               
     5                                               Args:
     6                                               video_path (str): Ruta al video a analizar.
     7                                               N (int): Cantidad de frames utilizados para calcular la estimación.
     8                                               recalc_interval (int): Intervalo de recálculo (medido en número de frames transcurridos).
     9                  

En este caso el algoritmo se aplica en la línea:

*fg_mask_mog2 = mog2.apply(frame)*

Si bien el tiempo utilizado es relativamente alto, es varios órdenes menor al cálculo de la mediana del algoritmo *naive*.

### Resumen y conclusiones

De los experimentos realizados se puede observar que tanto el algoritmo *naive* como MOG2 presentan resultados aceptables. En el caso *naive* se pueden apreciar ciertos artefactos no deseados cuando algunos objetos permanecen en la misma posición durante cierto tiempo, problema que no está tan presente en MOG2 debido a que el algoritmo es menos suceptible a estos cambios bruscos. De todas formas, ambos se favorecen de este video particular en que el fondo permanece bastante estático en todo momento.

Con respecto a performance, MOG2 consume más CPU pero de forma más distribuida que el caso *naive*, dado que este último tiene un costo muy grande cuando se calcula la mediana de los *N* frames. En cuanto al uso de memoria, el caso *naive* debe mantener siempre cierta cantidad de frames para poder tomar aleatoriamente *N* muestras mientras que MOG2 no, lo cual lo hace mucho más eficiente en este sentido.